In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso

In [2]:
import os
os.getcwd()

In [3]:
cs_train = pd.read_csv('counterfeit_train.csv')
cs_test = pd.read_csv('counterfeit_test.csv')

In [4]:
cs_test['Counterfeit_Sales']=np.nan  #Dummy column to identify test data separately
cs_train['data']='train' 
cs_test['data']='test' 
cs_test=cs_test[cs_train.columns]

cs_all=pd.concat([cs_train,cs_test],axis=0)

cs_all.drop(['Medicine_ID'],axis=1,inplace=True)

# EDA

In [5]:
cat_cols=cs_all.select_dtypes(['object']).columns[:-1]
print(cat_cols)

Index(['DistArea_ID', 'Medicine_Type', 'SidEffect_Level', 'Area_Type',
       'Area_City_Type', 'Area_dist_level'],
      dtype='object')


In [6]:
print(cs_test.shape)

(1705, 13)


In [7]:
cs_all.isnull().sum()

Counterfeit_Weight     1463
DistArea_ID               0
Active_Since              0
Medicine_MRP              0
Medicine_Type             0
SidEffect_Level           0
Availability_rating       0
Area_Type                 0
Area_City_Type            0
Area_dist_level           0
Counterfeit_Sales      1705
data                      0
dtype: int64

In [8]:
for col in cs_all.columns:
    if (col not in ['Counterfeit_Sales','data'])& (cs_all[col].isnull().sum()>0):
        cs_all.loc[cs_all[col].isnull(),col]=cs_all.loc[cs_all['data']=='train',col].mean()

In [9]:
cs_all.select_dtypes(['object']).columns

Index(['DistArea_ID', 'Medicine_Type', 'SidEffect_Level', 'Area_Type',
       'Area_City_Type', 'Area_dist_level', 'data'],
      dtype='object')

#### Convert variables with object datatype to dummy

In [10]:
for col in cat_cols:
    freqs=cs_all[col].value_counts()
    k=freqs.index[freqs>20][:-1]
    for cat in k:
        name=col+'_'+cat
        cs_all[name]=(cs_all[col]==cat).astype(int)
    del cs_all[col]
    print(col)

DistArea_ID
Medicine_Type
SidEffect_Level
Area_Type
Area_City_Type
Area_dist_level


In [11]:
cs_train=cs_all[cs_all['data']=='train']
del cs_train['data']
cs_test=cs_all[cs_all['data']=='test']
cs_test.drop(['Counterfeit_Sales','data'],axis=1,inplace=True)

del cs_all

C:\Anaconda\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [12]:
import pandas as pd
import math
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import mean_absolute_error
%matplotlib inline

In [13]:
from sklearn.model_selection import train_test_split
cs_train1,cs_train2=train_test_split(cs_train,test_size=0.2,random_state=2)

X_train=cs_train1.drop('Counterfeit_Sales',axis=1)
y_train=cs_train1['Counterfeit_Sales']

X_test=cs_train2.drop('Counterfeit_Sales',axis=1)
y_test=cs_train2['Counterfeit_Sales']

In [14]:
cs_train.shape

(6818, 38)

### Linear Regression

In [15]:
from sklearn.linear_model import LinearRegression

In [16]:
lm=LinearRegression()

In [17]:
lm.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [18]:
lm.intercept_

-0.014430681543672108

### For looking weightage for coefficients

In [19]:
list(zip(X_train.columns,lm.coef_))

[('Counterfeit_Weight', -4.761842500980404),
 ('Active_Since', -0.10517526530587494),
 ('Medicine_MRP', 15.426504040886151),
 ('Availability_rating', -273.65566358980544),
 ('DistArea_ID_Area027', 753.421779539505),
 ('DistArea_ID_Area013', -43.482848769757716),
 ('DistArea_ID_Area035', 210.58134615501788),
 ('DistArea_ID_Area049', -128.48741580975434),
 ('DistArea_ID_Area046', 377.13941744576005),
 ('DistArea_ID_Area045', -42.49319281086095),
 ('DistArea_ID_Area018', -233.75187976386064),
 ('DistArea_ID_Area017', 132.31729563724582),
 ('DistArea_ID_Area010', -247.66041189126602),
 ('Medicine_Type_Antibiotics', -291.0996042413327),
 ('Medicine_Type_Hreplacements', -389.8981854004221),
 ('Medicine_Type_Antiseptics', -385.50946491851704),
 ('Medicine_Type_OralContraceptives', -357.97743612048083),
 ('Medicine_Type_Antipyretics', -419.5937885595727),
 ('Medicine_Type_Cardiac', -280.4056157718869),
 ('Medicine_Type_Mstablizers', -371.3083520033227),
 ('Medicine_Type_Tranquilizers', -433.88

In [20]:
predicted=lm.predict(X_test)

In [21]:
mean_absolute_error(y_test,predicted)

830.050146841315

 MAE comes to 830 in this case.

# Lasso Model

In [22]:
lambdas=np.linspace(5,15,100)
params={'alpha':lambdas}

In [23]:
model=Lasso(fit_intercept=True)

In [24]:
lasmod=GridSearchCV(model,param_grid=params,cv=10,scoring='neg_mean_absolute_error')

In [25]:
lasmod.fit(X_train,y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'alpha': array([ 5.     ,  5.10101, ..., 14.89899, 15.     ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_absolute_error', verbose=0)

In [26]:
lasmod.best_estimator_

Lasso(alpha=8.333333333333334, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [27]:
lasso_model=lasmod.best_estimator_

In [28]:
lasso_model.fit(X_train,y_train)

Lasso(alpha=8.333333333333334, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [29]:
list(zip(X_train.columns,lasso_model.coef_))

[('Counterfeit_Weight', -4.597030359171696),
 ('Active_Since', -0.6942976854481621),
 ('Medicine_MRP', 15.414894718372992),
 ('Availability_rating', -0.0),
 ('DistArea_ID_Area027', 1572.5980867626008),
 ('DistArea_ID_Area013', -0.0),
 ('DistArea_ID_Area035', 40.86413506850696),
 ('DistArea_ID_Area049', 0.0),
 ('DistArea_ID_Area046', -0.0),
 ('DistArea_ID_Area045', -24.884775587596074),
 ('DistArea_ID_Area018', -0.0),
 ('DistArea_ID_Area017', 0.0),
 ('DistArea_ID_Area010', -0.0),
 ('Medicine_Type_Antibiotics', 3.337449383714278),
 ('Medicine_Type_Hreplacements', -0.0),
 ('Medicine_Type_Antiseptics', -0.0),
 ('Medicine_Type_OralContraceptives', -0.0),
 ('Medicine_Type_Antipyretics', -0.0),
 ('Medicine_Type_Cardiac', 0.0),
 ('Medicine_Type_Mstablizers', -0.0),
 ('Medicine_Type_Tranquilizers', -0.0),
 ('Medicine_Type_Analgesics', -0.0),
 ('Medicine_Type_Antimalarial', 0.0),
 ('Medicine_Type_Antacids', -0.0),
 ('Medicine_Type_Statins', 0.0),
 ('Medicine_Type_MuscleRelaxants', 0.0),
 ('Medic

In [30]:
lasso_pred=lasso_model.predict(X_test)

In [31]:
mean_absolute_error(y_test,lasso_pred)

824.6189488122499

MAE has slightly improved from 830 to 825 from linear regression to lasso regression

# RF

In [32]:
from sklearn.model_selection import RandomizedSearchCV

In [33]:
param_dist = {"n_estimators":[100,200,300,500,700,1000],
              "max_features": [5,10,20,25,30,35],
              "bootstrap": [True, False],
                'max_depth':[None,5,10,15,20,30,50,70],
                'min_samples_leaf':[1,2,5,10,15,20], 
                'min_samples_split':[2,5,10,15,20]
                  }

In [34]:
from sklearn.ensemble import RandomForestRegressor

In [35]:
rf=RandomForestRegressor()

In [36]:
ranfor=RandomizedSearchCV(rf,cv=10,
                                 param_distributions=param_dist,
                                 scoring='neg_mean_absolute_error',
                                 n_iter=10,verbose=10
                                    )

In [37]:
ranfor.fit(X_train,y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV] n_estimators=200, min_samples_split=20, min_samples_leaf=1, max_features=10, max_depth=50, bootstrap=True 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  n_estimators=200, min_samples_split=20, min_samples_leaf=1, max_features=10, max_depth=50, bootstrap=True, score=-762.1089841701751, total=   2.2s
[CV] n_estimators=200, min_samples_split=20, min_samples_leaf=1, max_features=10, max_depth=50, bootstrap=True 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.4s remaining:    0.0s


[CV]  n_estimators=200, min_samples_split=20, min_samples_leaf=1, max_features=10, max_depth=50, bootstrap=True, score=-781.5269337469547, total=   2.1s
[CV] n_estimators=200, min_samples_split=20, min_samples_leaf=1, max_features=10, max_depth=50, bootstrap=True 


...

[CV]  n_estimators=200, min_samples_split=20, min_samples_leaf=1, max_features=10, max_depth=50, bootstrap=True, score=-752.4241113965619, total=   2.1s
[CV] n_estimators=200, min_samples_split=5, min_samples_leaf=15, max_features=10, max_depth=30, bootstrap=True 
[CV]  n_estimators=200, min_samples_split=5, min_samples_leaf=15, max_features=10, max_depth=30, bootstrap=True, score=-762.3192812825444, total=   1.5s
[CV] n_estimators=200, min_samples_split=5, min_samples_leaf=15, max_features=10, max_depth=30, bootstrap=True 
[CV]  n_estimators=200, min_samples_split=5, min_samples_leaf=15, max_features=10, max_depth=30, bootstrap=True, score=-791.2777427820819, total=   1.5s
[CV] n_estimators=200, min_samples_split=5, min_samples_leaf=15, max_features=10, max_depth=30, bootstrap=True 
[CV]  n_estimators=200, min_samples_split=5, min_samples_leaf=15, max_features=10, max_depth=30, bootstrap=True, score=-801.0437450112836, total=   1.5s
[CV] n_estimators=200, min_samples_split=5, min_samp

[CV]  n_estimators=500, min_samples_split=5, min_samples_leaf=1, max_features=5, max_depth=5, bootstrap=False, score=-894.3201896683848, total=   1.9s
[CV] n_estimators=500, min_samples_split=5, min_samples_leaf=1, max_features=5, max_depth=5, bootstrap=False 
[CV]  n_estimators=500, min_samples_split=5, min_samples_leaf=1, max_features=5, max_depth=5, bootstrap=False, score=-948.2991413970032, total=   1.9s
[CV] n_estimators=500, min_samples_split=5, min_samples_leaf=1, max_features=5, max_depth=5, bootstrap=False 
[CV]  n_estimators=500, min_samples_split=5, min_samples_leaf=1, max_features=5, max_depth=5, bootstrap=False, score=-985.2466112348612, total=   1.9s
[CV] n_estimators=500, min_samples_split=5, min_samples_leaf=1, max_features=5, max_depth=5, bootstrap=False 
[CV]  n_estimators=500, min_samples_split=5, min_samples_leaf=1, max_features=5, max_depth=5, bootstrap=False, score=-917.3102533098811, total=   1.9s
[CV] n_estimators=500, min_samples_split=5, min_samples_leaf=1, ma

[CV]  n_estimators=300, min_samples_split=15, min_samples_leaf=10, max_features=10, max_depth=50, bootstrap=False, score=-800.5872241749362, total=   3.9s
[CV] n_estimators=300, min_samples_split=15, min_samples_leaf=10, max_features=10, max_depth=50, bootstrap=False 
[CV]  n_estimators=300, min_samples_split=15, min_samples_leaf=10, max_features=10, max_depth=50, bootstrap=False, score=-767.9801738218728, total=   3.9s
[CV] n_estimators=300, min_samples_split=15, min_samples_leaf=10, max_features=10, max_depth=50, bootstrap=False 
[CV]  n_estimators=300, min_samples_split=15, min_samples_leaf=10, max_features=10, max_depth=50, bootstrap=False, score=-719.1085829017536, total=   4.2s
[CV] n_estimators=300, min_samples_split=15, min_samples_leaf=10, max_features=10, max_depth=50, bootstrap=False 
[CV]  n_estimators=300, min_samples_split=15, min_samples_leaf=10, max_features=10, max_depth=50, bootstrap=False, score=-733.3598774920719, total=   4.0s
[CV] n_estimators=300, min_samples_spl

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 10.7min finished


RandomizedSearchCV(cv=10, error_score='raise-deprecating',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=10, n_jobs=None,
          param_distributions={'n_estimators': [100, 200, 300, 500, 700, 1000], 'max_features': [5, 10, 20, 25, 30, 35], 'bootstrap': [True, False], 'max_depth': [None, 5, 10, 15, 20, 30, 50, 70], 'min_samples_leaf': [1, 2, 5, 10, 15, 20], 'min_samples_split': [2, 5, 10, 15, 20]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='neg_mean_absolute_error',
          verbose=10)

In [38]:
random_forest = ranfor.best_estimator_

In [39]:
random_forest.fit(X_train,y_train)

RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=50,
           max_features=10, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=10,
           min_samples_split=15, min_weight_fraction_leaf=0.0,
           n_estimators=300, n_jobs=None, oob_score=False,
           random_state=None, verbose=0, warm_start=False)

In [40]:
ranfor_pred=random_forest.predict(X_test)

In [41]:
mean_absolute_error(y_test,ranfor_pred)

777.4032375871675

# GBM

In [42]:
from sklearn.ensemble import GradientBoostingRegressor

In [43]:
gbm_params = {'n_estimators' : [80,90,100,110,120],
'learning_rate' : [0.2,0.3,0.4,0.5,0.6],
'max_depth' : [2,3],
'subsample' : [0.5,0.8,1],
'max_features' : [18,19,20,21,22] }

In [44]:
gbm = GradientBoostingRegressor()

In [45]:
gbm_search = RandomizedSearchCV(gbm,scoring='neg_mean_absolute_error',param_distributions=gbm_params,
                                cv=10, n_iter=10, n_jobs=-1)

In [46]:
gbm_search.fit(X_train,y_train)

RandomizedSearchCV(cv=10, error_score='raise-deprecating',
          estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_sampl...=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'n_estimators': [80, 90, 100, 110, 120], 'learning_rate': [0.2, 0.3, 0.4, 0.5, 0.6], 'max_depth': [2, 3], 'subsample': [0.5, 0.8, 1], 'max_features': [18, 19, 20, 21, 22]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='neg_mean_absolute_error',
          verbose=0)

In [47]:
gbm_search.best_estimator_.fit(X_train,y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.2, loss='ls', max_depth=3, max_features=22,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=110, n_iter_no_change=None, presort='auto',
             random_state=None, subsample=1, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [48]:
gbm_pred=gbm_search.predict(X_test)

In [49]:
mean_absolute_error(y_test,gbm_pred)

781.3290849725605

# Stacking

In [50]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold

In [51]:
clf1=Lasso(alpha=8.333333333333334, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)
clf2=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=70,
           max_features=25, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=20,
           min_samples_split=10, min_weight_fraction_leaf=0.0,
           n_estimators=500, n_jobs=None, oob_score=False,
           random_state=None, verbose=0, warm_start=False)
clf3=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.5, loss='ls', max_depth=2, max_features=18,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=80, n_iter_no_change=None, presort='auto',
             random_state=None, subsample=0.8, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

Algos=[clf1,clf2,clf3]

In [52]:
rows=X_train.shape[0]

In [53]:
layer1=pd.DataFrame({'clf1':np.zeros(rows),'clf2':np.zeros(rows),'clf3':np.zeros(rows)})

In [54]:
kf=KFold(n_splits=10)

In [55]:
X_train.reset_index(drop=True,inplace=True )
y_train.reset_index(drop=True,inplace=True )

In [56]:
fold=1
for train,left_out_chunk in kf.split(X_train):
    print('fold number : ', fold)
    
    for i,clf in enumerate(Algos):
        print('Algo number :',i+1)
        
        X_train_train=X_train.loc[train]
        y_train_train=y_train[train]
        X_train_left_out_chunk=X_train.loc[left_out_chunk]
        
        clf.fit(X_train_train,y_train_train)
        p=clf.predict(X_train_left_out_chunk)#[:,1]
        
        layer1.iloc[left_out_chunk,i]=p
        
    fold+=1  
    

fold number :  1
Algo number : 1
Algo number : 2
Algo number : 3
fold number :  2
Algo number : 1
Algo number : 2
Algo number : 3
fold number :  3
Algo number : 1
Algo number : 2
Algo number : 3
fold number :  4
Algo number : 1
Algo number : 2
Algo number : 3
fold number :  5
Algo number : 1
Algo number : 2
Algo number : 3
fold number :  6
Algo number : 1
Algo number : 2
Algo number : 3
fold number :  7
Algo number : 1
Algo number : 2
Algo number : 3
fold number :  8
Algo number : 1
Algo number : 2
Algo number : 3
fold number :  9
Algo number : 1
Algo number : 2
Algo number : 3
fold number :  10
Algo number : 1
Algo number : 2
Algo number : 3


In [57]:
rows=X_test.shape[0]
layer2_test=pd.DataFrame({'clf1':np.zeros(rows),'clf2':np.zeros(rows),'clf3':np.zeros(rows)})

In [58]:
for i,clf in enumerate(Algos):
    print( 'Algo number',i+1)
    clf.fit(X_train,y_train)
    p=clf.predict(X_test)
    
    layer2_test.iloc[:,i]=p


Algo number 1
Algo number 2
Algo number 3


In [59]:
from sklearn.linear_model import LinearRegression

In [60]:
# second layer linear model 
lmr=LinearRegression()

In [61]:
X_test.shape

(1364, 37)

In [62]:
lmr.fit(layer1,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [63]:
from sklearn.metrics import mean_absolute_error

In [64]:
pred = lmr.predict(layer2_test)

In [65]:
mean_absolute_error(y_test,pred)

764.2121640962251

#### We can conclude that all the models together find different patterns and helps in reducing the error by around 66.